# SQream DB and H2o.ai Demo

This demonstration shows how to use H2O's AI platform, including H2O Flow, with SQream DB.

The first half of this demo is standard Python inside a Jupyter Notebook, the other half is in H2O's Flow UI.

We are running Python 3.7 on a standard laptop.

## Step 1 - import and initialize H2O and SQream DB connectors

In [ ]:
import os
import h2o
import pysqream as sq
import matplotlib.pyplot as plt

In [ ]:
h2o.init(url='http://127.0.0.1:54321', nthreads=-1, bind_to_localhost=False)

In [ ]:
con = sq.connect(host='192.168.1.91',port=5000
                 ,database='ontime'
                 ,username=os.environ.get("SQREAM_USERNAME")
                 ,password=os.environ.get("SQREAM_PASSWORD")
                 ,clustered=False)

Success, we are connected to SQream DB!

## Step 2 - Exploring the data a bit

Let us run a couple of queries to see what we have in this `ontime` database

In [ ]:
cur = con.cursor() # We create a new database cursor
cur.execute("SELECT table_name, row_count FROM sqream_catalog.tables")
print(cur.fetchall())
cur.close()

Our database has one table called `ontime`, with 193.1 million records.

Let's run a query on this table, to see how many distinct years it contains...

In [ ]:
cur = con.cursor() # We create a new database cursor
cur.execute("SELECT year, COUNT(1) FROM ontime GROUP BY 1")
result = cur.fetchall()
cur.close()
result

Alright, entries from 1987 all the way to 2020.
Let's plot it!

In [ ]:
plt.bar(*zip(*result))
plt.show()

So far, run-of-the-mill Python.

## Step 3 - adding calculated columns for prediction

Let's try to do something more interersting.

Our table includes information about every flight in the US from 1987, until January 2020.

We want to know which flights were delayed. Let's define a delay:

> a departure delay is whenever the flight was delayed on departure for over 15 minutes

We can ask SQream DB to attach a new calculated column, `isDepDelayed` by adding it to the end of the select list.

In this case, let's look at 1000 flights in 2005:

In [ ]:
query = """SELECT *,coalesce((depdelay > 15),false) AS isdepdelayed
FROM ontime
WHERE year = 2005
LIMIT 1000
"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchall()
cur.close()

In [ ]:
# Take result and place it in an H2O data frame:
airlines_dataset = h2o.H2OFrame(result,column_names=[ i[0] for i in cur.description ], destination_frame="airlines_training")
airlines_dataset.head(10)

Once we have a dataframe, we can begin to feed the data into a model.

## Step 4 - build the model

First, we decide on some columns which will be the "dimensions" (called factors in H2O).

In [ ]:
def factorize(df, cnames):
    for cname in cnames:
        df[cname] = df[cname].asfactor()
    return df

# We can tell H2O which columns we consider to be factors:
airlines_dataset = factorize(airlines_dataset, 
                             ["year","month","dayofmonth"
                              ,"dayofweek","uniquecarrier"
                              ,"origin","dest"
                              ,"isdepdelayed"])

We will use a Gradient Boosting Estimator (GBM).

First, we'll want to predict the "is departure delayed" column that we added. So, that column will be separated from the rest:

In [ ]:
predict_column = "isdepdelayed" # Column to predict

other_cols = ['origin','dest','dayofmonth'
              ,'year','uniquecarrier','dayofweek'
              ,'month','deptime','crsdeptime'
              ,'arrtime','distance'] # Other factors to look at

Now, we'll take our data frame and split it 80%/20%.

At 1000 rows for the original training set, we're looking at around 800 rows for the training set, and 200 for the validation set - just to check that our model works correctly.

In [ ]:
train, valid = airlines_dataset.split_frame([0.8,], seed=1234)

Now, we create the GBM:

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
gbm_v1 = H2OGradientBoostingEstimator(model_id="gbm_airlines", seed=2000000)

## Step 5 - Train the model

In [ ]:
gbm_v1.train(other_cols, predict_column, training_frame=train, validation_frame=valid)

## Step 6 - Inspect the model in H2O Flow

We can now look at the model in H2O Flow: http://127.0.0.1:54321/flow/index.html

## Step 7 - Test the model

In order to test the model, we'll pull out 10,000 rows from the year 2019 and 2018 and feed it into H2O.

Then, we'll try to predict, but we'll do it in the H2O Flow UI.

In [ ]:
query = """SELECT year,month,dayofmonth
                   ,dayofweek,uniquecarrier
                   ,origin,dest
                   ,coalesce((depdelay > 15),false) as isdepdelayed
           FROM ontime WHERE year=2019
           LIMIT 10000
"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchall()
test = h2o.H2OFrame(result,column_names=[ i[0] for i in cur.description ],destination_frame="airlines_test_2019")
cur.close()

In [ ]:
query = """SELECT year,month,dayofmonth
                  ,dayofweek,uniquecarrier
                  ,origin,dest
                  ,coalesce((depdelay > 15),false) as isdepdelayed
           FROM ontime WHERE year=2018
           LIMIT 10000
"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchall()
test = h2o.H2OFrame(result,column_names=[ i[0] for i in cur.description ],destination_frame="airlines_test_2018")
cur.close()